In [1]:
import pandas as pd
import numpy as np
import src.in_out as io
import src.info as info
import src.condition as con
import src.base as base
from itertools import combinations
from src.spread import get_spread
import src.change as change

## Veri Hazırlama

In [2]:
#Verilere ait path'ler alınıyor
path_list=io.get_path('data')

In [3]:
#Okunacak sütun adları
cols= ["Time","BID price","ASK price"]
#Tüm veriler okunuyor
master_data=io.read_allData(path_list, cols=cols)
master_data.columns=['time','bid_price','ask_price','name','date']

In [4]:
#date sütunu Timestamp'e dönüştürülüyor ve tarih ve saat bilgisi birleştiriliyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [5]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

## Mid Price

In [6]:
# mid price hesaplanıyor
master_data['mid_price']=base.average_of_series(master_data.bid_price,master_data.ask_price)

In [7]:
# spread hesaplama için master_data, uygun bir forma dönüştürülüyor
df_mid_price=master_data.pivot(index='date', columns='name', values='mid_price')

## Spread
<img src="img/spread_flowchart.png">
<br/>
Spread, her bir saat için ayrı ayrı hesaplanmıştır

In [8]:
#Ticksize ve pnltick değerleri okunuyor
tick_size=pd.read_excel('ticksize.xlsx',index_col='property')

In [15]:
# pair'lar alınıyor
pairs=combinations(df_mid_price.columns, 2)

In [13]:
#spread hesaplanıyor
spread_list=[
    base.groupby_date_time(df_mid_price.loc[:,pair],hour='1h').apply(lambda x: get_spread(x,tick_size.loc[:,pair]))\
        for pair in pairs]
df_spread=pd.concat(spread_list,axis=1).droplevel(0)

In [16]:
#spread hesaplanıyor
spread_list=[
    base.groupby_date_time(df_mid_price.loc[:,pair],day='1d',hour='1h').apply(lambda x: get_spread(x,tick_size.loc[:,pair]))\
        for pair in pairs]
df_spread=pd.concat(spread_list,axis=1).droplevel([0,1])

## Change

In [18]:
#change alınıyor
df_change=base.groupby_date_time(df_spread,day='1d',hour='1h').apply(change.get_change)
df_change=df_change.droplevel([0,1])

## Amplitude

In [19]:
#Hesaplanan amplitude'ları saklamak için boş bir dizi oluşturduk.
all_pairs_of_amplitude=[]

In [20]:
for pair_change in df_change:   #Her bir pair alınıyor
    pair_group = list(base.groupby_date_time(df_change[pair_change],day='1d',hour='1h')) #gün ve saate göre gruplanıyor
    sub_amplitudes=[]
    for i in range(len(pair_group)):
        sub_change=base.to_series(pair_group[i][1])
        sub_amplitudes.append(change.get_amplitude(sub_change)) #pair'a ait her alt grubun amplitudu'u hesaplanıyor
    all_pairs_of_amplitude.append(pd.concat(sub_amplitudes)) # alt amplitude'lar birleştiriliyor

#### NOT : Amplitude'u hesaplanan tüm pair'lar all_pairs_of_amplitude adlı dizide tutuluyor.  Hepsi tek DataFrame'de birleştirilmek istenirse aşağıdaki kod çalıştırılabilir.

In [21]:
from functools import reduce

df_final = reduce(lambda left,right: pd.merge(left,right,on=['date','duration'],how='outer'), all_pairs_of_amplitude)

df_final # NaN'lar birleştirmeden dolayı. ÖNEMSİZ ! 6AU8_6CU8, 2018-09-06 00:00:49 dan başladığından öncesi NaN

,date,duration,6AU8_6BU8,6AU8_6CU8,6BU8_6CU8
0,2018-09-06 00:00:17,00:00:13,-0.000145,NaN,0.000145
1,2018-09-06 00:00:39,00:00:22,0.000025,NaN,-0.000037
2,2018-09-06 00:00:48,00:00:09,-0.000039,NaN,0.000039
3,2018-09-06 00:00:49,00:00:01,0.000023,-0.000014,-0.000037
4,2018-09-06 00:00:50,00:00:01,-0.000023,0.000014,0.000037
...,...,...,...,...,...
12136,2018-09-07 23:54:53,00:00:02,NaN,NaN,0.000042
12137,2018-09-07 23:55:36,00:00:01,NaN,NaN,0.000037
12138,2018-09-07 23:55:54,00:00:17,NaN,NaN,0.000067
12139,2018-09-07 23:58:54,00:00:04,NaN,NaN,-0.000071


# Koşula Göre Amplitude ve Duration Verisinin Yeniden Düzenlenmesi

In [22]:
#Önce condition modülümüzü import ediyoruz.
import src.condition as condition

In [23]:
# Hesaplayacağımız verileri saklayabilmek için boş bir dizi yarattık
condition_data = []

In [24]:
# Python'a ait lambda ifadelerini kullanabilmek için basit bir fonksiyon yazdık
def uygula(data):
    """parametre olarak gelen 'data' daki indeksi kaldırır.
       percentile(.6) ya göre koşul uygulandı.
       percentile silinip yerine medyan() yazılırsa, koşulu medyana göre hesaplar.
       ya da percentile'ın '.6' olan parametresi değiştirilebilir.
    """
    data = data.reset_index(drop=True)
    signs = condition.Sign(data).percentile(.6) #
    return condition.apply(data,signs)

In [25]:
# bir for döngüsü ile tüm pairlara erişiyoruz
for data in all_pairs_of_amplitude:
    #groupby içerisindeki floor metoduyla veriyi neye göre gruplayacağımızı belirtiyoruz.
    #Burada groupby 2 parametre aldı. ilk parametre ile 'güne göre', 2. parametre ile de 'saate' göre
    #ayırıp, 'uygula' metodunu her gruba ayrı ayrı uygulayacağını belirttik
    """ 'd' -> day
        '2h' -> 2şer saat
        'h'-> birer saat
        'nd' ->n güne böl
        'nh' -> n saate böl gibi        
    """ 
    """.apply metodu ile yapmak istediğimiz işlemi belirtiyoruz
        lambda x : uygula(x) -> her bir grubu sırayla
        x olarak adlandır ve uygula fonksiyonuna parametre olarak gönder
    """
    #Sadece saate göre yapılmak istenirse
#     result = data.groupby([data.date.dt.floor('h')])\
#         .apply(lambda x : uygula(x)).droplevel([0,1])
    result = base.groupby_date_time(data,day='1d',hour='1h')\
        .apply(lambda x : uygula(x)).droplevel([0,1])
    #son olarak hesaplanan verileri condition_data adlı listeye atmasını söylüyoruz
    condition_data.append(result)

In [26]:
# 1. pairi görüntüleyelim

In [27]:
condition_data[0]

,date,duration,6AU8_6BU8
0,2018-09-06 00:00:17,00:00:13,-0.000145
1,2018-09-06 00:00:39,00:00:22,0.000025
2,2018-09-06 00:00:53,00:00:14,-0.000073
3,2018-09-06 00:01:01,00:00:08,0.000057
4,2018-09-06 00:01:22,00:00:21,-0.000106
...,...,...,...
197,2018-09-07 23:56:50,00:00:16,-0.000031
198,2018-09-07 23:57:25,00:00:35,0.000031
199,2018-09-07 23:58:50,00:01:25,-0.000133
200,2018-09-07 23:58:52,00:00:02,0.000067


## Aralık Oluşturma

Pair'lerden biri olan **6AU8_6BU8**'deki verileri aralığa yerleştirelim. Bunun için **interval** modülündeki **set_range** ve **set_timeRange** fonksiyonlarını kullanacağız.
<span style="text-decoration:underline">amplitude</span> için **set_range**, <span style="text-decoration:underline">duration</span> için ise **set_timeRange** kullanacağız. Duration'nın veri tipi **timedelta64** olduğundan ayrı bir metot yazıldı.<br/>
Fonksiyonlar **data**, **value**, ve **method** olmak üzere 3 parametre alarak çalışır. Parametreler hakkında kısa bilgi:
1. **data** : aralığa yerleştirilecek veriler. örn. duration veya amplitude
2. **method** : aralık oluşturma yöntemini burada belirtiyoruz. **2** çeşit aralık oluşturma yöntemi var. **'width'** ve **'size'**
   *  **width** : yöntemi seçilirse her bir aralığın genişliği **value** değeri kadardır.
   *  **size** : yöntemi seçilirse **@value** adet aralık oluşturur. Her aralığın genişliği aynıdır.
   *  **value** : yöntem 'width' ise aralık genişliğini, 'size' ise aralık adetini belirtir.

**6AU8_6BU8** pair'ini alalım. Duration'ı **5'er sn'lik aralıklara** yerleştirelim. Amplitude verisini ise **aralık sayısı 100 olacak** ve **her aralığın genişliği eşit** olacak şekilde aralıklara yerleştirelim. Böylece her iki yöntemi de görmüş olacağız

In [56]:
# 6AU8_6BU8, condition_data adlı dizinin 0. elemanı
ab = condition_data[0]
ab

,date,duration,6AU8_6BU8
0,2018-09-06 00:00:17,00:00:13,-0.000145
1,2018-09-06 00:00:39,00:00:22,0.000025
2,2018-09-06 00:00:53,00:00:14,-0.000073
3,2018-09-06 00:01:01,00:00:08,0.000057
4,2018-09-06 00:01:22,00:00:21,-0.000106
...,...,...,...
197,2018-09-07 23:56:50,00:00:16,-0.000031
198,2018-09-07 23:57:25,00:00:35,0.000031
199,2018-09-07 23:58:50,00:01:25,-0.000133
200,2018-09-07 23:58:52,00:00:02,0.000067


In [57]:
#6AU8_6BU8 'den date,duration ve amplitude sütunları alınıyor
date = ab['date'].reset_index(drop=True)
duration = ab['duration']
amplitude = ab['6AU8_6BU8']

In [58]:
# interval modülünden, kullanacağımız fonksiyonları import ediyoruz
from src.interval import set_range, set_timeRange

In [59]:
# duration için her biri 5'er sn'lik aralık oluşturuyoruz.
new_duration=set_timeRange(data=duration, value=5, method='width')
new_duration

0       (0 days 00:00:10, 0 days 00:00:15]
1       (0 days 00:00:20, 0 days 00:00:25]
2       (0 days 00:00:10, 0 days 00:00:15]
3       (0 days 00:00:05, 0 days 00:00:10]
4       (0 days 00:00:20, 0 days 00:00:25]
                       ...                
4445    (0 days 00:00:15, 0 days 00:00:20]
4446    (0 days 00:00:30, 0 days 00:00:35]
4447    (0 days 00:01:20, 0 days 00:01:25]
4448    (0 days 00:00:00, 0 days 00:00:05]
4449    (0 days 00:00:10, 0 days 00:00:15]
Name: duration, Length: 4450, dtype: category
Categories (88, interval[timedelta64[ns]]): [(0 days 00:00:00, 0 days 00:00:05] < (0 days 00:00:05, 0 days 00:00:10] < (0 days 00:00:10, 0 days 00:00:15] < (0 days 00:00:15, 0 days 00:00:20] ... (0 days 00:07:00, 0 days 00:07:05] < (0 days 00:07:05, 0 days 00:07:10] < (0 days 00:07:10, 0 days 00:07:15] < (0 days 00:07:15, 0 days 00:07:20]]

In [60]:
# amplitude için her biri eşit büyüklükte 300 adet aralık oluşturuldu ve veriler aralıklara yerleştirildi
new_amplitude = set_range(data=amplitude, value=300, method='size')
new_amplitude

0       (-0.00343, 8.05e-05]
1       (-0.00343, 8.05e-05]
2       (-0.00343, 8.05e-05]
3       (-0.00343, 8.05e-05]
4       (-0.00343, 8.05e-05]
                ...         
4445    (-0.00343, 8.05e-05]
4446    (-0.00343, 8.05e-05]
4447    (-0.00343, 8.05e-05]
4448    (-0.00343, 8.05e-05]
4449    (-0.00343, 8.05e-05]
Name: 6AU8_6BU8, Length: 4450, dtype: category
Categories (299, interval[float64]): [(-0.463, -0.46] < (-0.46, -0.456] < (-0.456, -0.453] < (-0.453, -0.449] ... (0.572, 0.576] < (0.576, 0.579] < (0.579, 0.583] < (0.583, 0.586]]

In [61]:
#pandas'ın concat metodunu kullarak hesapladığımız aralıkları DataFrame'de gösterdik
pd.concat([new_duration,new_amplitude],axis=1)

,duration,6AU8_6BU8
0,"(0 days 00:00:10, 0 days 00:00:15]","(-0.00343, 8.05e-05]"
1,"(0 days 00:00:20, 0 days 00:00:25]","(-0.00343, 8.05e-05]"
2,"(0 days 00:00:10, 0 days 00:00:15]","(-0.00343, 8.05e-05]"
3,"(0 days 00:00:05, 0 days 00:00:10]","(-0.00343, 8.05e-05]"
4,"(0 days 00:00:20, 0 days 00:00:25]","(-0.00343, 8.05e-05]"
...,...,...
4445,"(0 days 00:00:15, 0 days 00:00:20]","(-0.00343, 8.05e-05]"
4446,"(0 days 00:00:30, 0 days 00:00:35]","(-0.00343, 8.05e-05]"
4447,"(0 days 00:01:20, 0 days 00:01:25]","(-0.00343, 8.05e-05]"
4448,"(0 days 00:00:00, 0 days 00:00:05]","(-0.00343, 8.05e-05]"


Yukarıda görüldüğü gibi verilerimizi aralığa yerleştirdik. Fakat her biri Interval türünden olduğundan dolayı hareket alanımızı kısıtlayacaktır.
Örneğin aşağıdaki gibi sütun verileriyle bir toplama işlemi gerçekleştirmek istediğimizi varsayalım :
```python
new_amplitude.sum()
```
Bu hamlemiz aşağıdaki hata ile sonuçlanacktır:
```
TypeError: Categorical cannot perform the operation sum
```
Aralık tanımlamadaki amaç, bu verileri **kategorilendirmek** olduğundan, her bir aralığın yalnızca **sağ değeri** ya da **ortalamasıyla** çalışmamız işlemlerimizin sonucunu etkilemeyecektir. Bize istatistiksel hesaplamalar veya grafik işlemlerinde kolaylık sağlayacaktır.<br/>

Şimdi duration ve amplitude aralıklarının ortalamasını aşağıdaki gibi alalım. Önce **interval** modülümüzden **get_mid** fonksiyonumuzu import edelim.

In [62]:
from src.interval import get_mid

In [63]:
#Amplitude için aralıkların ortalamasını al ve veriyi float'a dönüştür
new_amplitude=get_mid(new_amplitude).astype('float64')

In [64]:
#Duration için aralıkların ortalamasını al ve veriyi timedelta64'e dönüştür
new_duration=get_mid(new_duration).astype('timedelta64')

In [65]:
#Verileri  DataFrame ile gösterelim ve 'new_ab' adlı değişkende saklayalım
new_ab=pd.concat([date,new_duration,new_amplitude],axis=1)
new_ab

,date,duration,6AU8_6BU8
0,2018-09-06 00:00:17,00:00:12.500000,-0.001675
1,2018-09-06 00:00:39,00:00:22.500000,-0.001675
2,2018-09-06 00:00:53,00:00:12.500000,-0.001675
3,2018-09-06 00:01:01,00:00:07.500000,-0.001675
4,2018-09-06 00:01:22,00:00:22.500000,-0.001675
...,...,...,...
4445,2018-09-07 23:56:50,00:00:17.500000,-0.001675
4446,2018-09-07 23:57:25,00:00:32.500000,-0.001675
4447,2018-09-07 23:58:50,00:01:22.500000,-0.001675
4448,2018-09-07 23:58:52,00:00:02.500000,-0.001675


Aralık işlemleri bu kadar. Şimdi oluşturduğumuz **new_ab** ile **Density İşlemleri**'ni gerçekleştireceğiz.

## Density

### 1. Joint Density
Bu bölümde aşağıdaki işlemler yapılmaktadır:
1. **get_frequency:** Her bir  {duration,amplitude} değerinden kaç tane olduğunu bulma işlemidir. Örneğin duration'ı 5 amplitude'u 10 olan 133 veri var, duration:5 amplitude:7 olan 23 veri var. **Parametre olarak** yalnızca **duration** ve **amplitude** içeren bir **DataFrame** alır. 
2. **joint_density:** Her bir frekansın, toplam frekansa bölünerek tabloda ilgili yere yerleştirilmiş halidir.Daha sonra bu tablonun yatay ve dikey toplamlarını alarak **marginal density** elde edilecektir. **Parametre olarak** yalnızca **duration** ve **amplitude** içeren bir **DataFrame** alır. 

**density** modülünden kullanacağımız fonksiyonları import edelim ve işlemleri gerçekleştirelim

In [72]:
from src.density import get_frequency, joint_density

In [73]:
# frekans hesaplanıyor
get_frequency(new_ab)

,duration,6AU8_6BU8,frequency
0,00:00:02.500000,-0.001675,1199
1,00:00:07.500000,-0.001675,837
2,00:00:12.500000,-0.001675,523
3,00:00:17.500000,-0.001675,327
4,00:00:22.500000,-0.001675,232
...,...,...,...
73,00:00:02.500000,0.156000,1
74,00:00:02.500000,0.131500,1
75,00:00:02.500000,0.040450,1
76,00:00:02.500000,0.033450,1


In [74]:
#joint density hesaplanıyor ve joint adlı bir değişkende saklanıyor
joint=joint_density(new_ab)
joint

6AU8_6BU8,-0.265000,-0.156000,-0.040250,-0.005185,-0.001675,0.001835,0.033450,0.040450,0.131500,0.156000,0.584500
duration,,,,,,,,,,,
00:00:02.500000,NaN,NaN,0.000226,NaN,0.270655,0.031828,0.000226,0.000226,0.000226,0.000226,0.000226
00:00:07.500000,0.000226,0.000226,NaN,NaN,0.188939,0.032506,NaN,NaN,NaN,NaN,NaN
00:00:12.500000,NaN,NaN,NaN,NaN,0.118059,0.020316,NaN,NaN,NaN,NaN,NaN
00:00:17.500000,NaN,NaN,NaN,NaN,0.073815,0.013318,NaN,NaN,NaN,NaN,NaN
00:00:22.500000,NaN,NaN,NaN,NaN,0.052370,0.006546,NaN,NaN,NaN,NaN,NaN
00:00:27.500000,NaN,NaN,NaN,0.000226,0.028442,0.004063,NaN,NaN,NaN,NaN,NaN
00:00:32.500000,NaN,NaN,NaN,NaN,0.030474,0.003612,NaN,NaN,NaN,NaN,NaN
00:00:37.500000,NaN,NaN,NaN,NaN,0.019413,0.002709,NaN,NaN,NaN,NaN,NaN
00:00:42.500000,NaN,NaN,NaN,NaN,0.017381,0.002709,NaN,NaN,NaN,NaN,NaN


Şimdi bulduğumuz **joint density**'nin **yataydaki ve dikeydeki toplamını** alalım. Fakat bulacağımız toplamları, aşağıdaki **Marjinal Density** tablosundaki gibi, tekrardan joint dataframe'ine aktarmayacağız. **vertical_total** ve **horizontal_total** adlı iki değişkende tutacağız. Daha sonra ise bunları kullanarak **Conditional Density** hesaplayacağız.
```
Örnek Tablo
| xy           | 4             | 6             | 8             | yatay toplam |
|--------------|---------------|---------------|---------------|--------------|
| 1            | 0,07547169811 | 0,2830188679  | 0,1320754717  | 0,4905660377 |
| 2            | 0,05031446541 | 0,07547169811 | 0,1446540881  | 0,2704402516 |
| 3            | 0,1257861635  | 0,04402515723 | 0,06918238994 | 0,2389937107 |
| dikey toplam | 0,251572327   | 0,4025157233  | 0,3459119497  | 1            |
```
Ayrıca aşağıda bulacağımız **vertical_total** ve **horizontal_total**'in toplamı **1'e eşittir**.

In [ ]:
# Dikey toplam
vertical_total = joint.agg('sum') #agg metodu pandas'a ait
vertical_total

In [ ]:
#Yatay toplam
horizontal_total = joint.agg('sum',axis=1)
horizontal_total

### 2. Conditional Density
Conditional Density 2 şekilde hesaplanır. x eksenine veya y eksenine göre(x:duration, y:amplitude temsil eder).<br/>
1. **Conditional Density / y** için: joint density'deki(**joint**) her değer, karşılık gelen dikey toplama(**vertical_total**) bölünerek hesaplanır.
2. **Conditional Density / x** için: joint density'deki(**joint**) her değer, karşılık gelen yatay toplama(**horizontal_total**) bölünerek hesaplanır.
```
Örnek Tablo (Conditional Density / y)
| xy | 4   | 6        | 8        |
|----|-----|----------|----------|
| 1  | 0,3 | 0,703125 | 0,381... |
| 2  | 0,2 | 0,1875   | 0,418..  |
| 3  | 0,5 | 0,109375 | 0,2      |
|    | 1   | 1        | 1        |
```

In [ ]:
# Conditional Density / y
(joint/vertical_total)

In [ ]:
# Conditional Density / x
(joint.T/horizontal_total)

# Veriyi belirli zaman dilimlerine göre ayırarak çalışmak

Önce **density** modülümüzden **grupla** fonksiyonumuzu import ediyoruz.

In [ ]:
from src.density import grupla

Aşağıdaki veriden
1. belirli bir saate göre
2. belirli günlere göre
3. hem güne hem de saate göre
veri çıkaracağız

In [ ]:
new_ab

### 1. Belirli Bir Saat Dilimindeki Veriler ile Çalışmak

In [ ]:
ab_grup = grupla(new_ab,hour='2h')

**hour** parametresini **'2h'** olarak girdik. Verileri **2'şer saate böldü** 

In [ ]:
#00:00:00-01:59:59 arasını aşağıdaki gibi alabiliriz. Bunun için get_group kullanacağız
ab_grup.get_group(0)